# Análisis exploratorio y descripción general de los datos a usar (RawData)

## Introducción

(Añadir descripción de cómo usar DataFiltering)

## 0. Cargar librerías

In [1]:
import pandas as pd
import numpy as np
import time as time
import dask.dataframe as dd
from tqdm import tqdm

import functions as fn

## 1. Cargar el dataset de fallecidos por COVID-19

### Cargar el dataset

In [2]:
url_fal = 'RawData/fallecidos_covid.csv'
df_fal = pd.read_csv(url_fal, sep = ";", encoding= "utf8", low_memory=False)

del url_fal
df_fal.head()

,FECHA_CORTE,FECHA_FALLECIMIENTO,EDAD_DECLARADA,SEXO,CLASIFICACION_DEF,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,id_persona
0,20220101,20211127,77,MASCULINO,Criterio virolÃ³gico,ANCASH,SANTA,NEPEÃA,021806,25819410.0
1,20220101,20210425,70,MASCULINO,Criterio virolÃ³gico,LIMA,LIMA,VILLA MARIA DEL TRIUNFO,150143,24693187.0
2,20220101,20210428,70,MASCULINO,Criterio virolÃ³gico,LIMA,LIMA,SAN JUAN DE MIRAFLORES,150133,24693271.0
3,20220101,20210430,59,FEMENINO,Criterio virolÃ³gico,AYACUCHO,HUANTA,HUANTA,050401,24735448.0
4,20220101,20210421,55,MASCULINO,Criterio SINADEF,LIMA,LIMA,ATE,150103,24786629.0


### Información general del dataset

In [6]:
df_fal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202782 entries, 0 to 202781
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   FECHA_CORTE          202782 non-null  int64  
 1   FECHA_FALLECIMIENTO  202782 non-null  int64  
 2   EDAD_DECLARADA       202782 non-null  int64  
 3   SEXO                 202782 non-null  object 
 4   CLASIFICACION_DEF    202782 non-null  object 
 5   DEPARTAMENTO         202782 non-null  object 
 6   PROVINCIA            202777 non-null  object 
 7   DISTRITO             202777 non-null  object 
 8   UBIGEO               202782 non-null  object 
 9   id_persona           199373 non-null  float64
dtypes: float64(1), int64(3), object(6)
memory usage: 15.5+ MB


### Porcentaje de datos perdidos de cada variable

In [7]:
fn.missing_values(df_fal)

,index,percent_missing
0,FECHA_CORTE,0.000000
1,FECHA_FALLECIMIENTO,0.000000
2,EDAD_DECLARADA,0.000000
3,SEXO,0.000000
4,CLASIFICACION_DEF,0.000000
5,DEPARTAMENTO,0.000000
6,PROVINCIA,0.002466
7,DISTRITO,0.002466
8,UBIGEO,0.000000
9,id_persona,1.681116


### Analizar cada variable del dataset de fallecidos

#### 1.1. FECHA_CORTE

In [8]:
print('Fecha de corte:')
df_fal['FECHA_CORTE'].value_counts().sort_index()

Fecha de corte:


20220101    202782
Name: FECHA_CORTE, dtype: int64

Todos los valores son iguales. La misma fecha de corte para n número de casos, variable no necesaria.

#### 1.2. FECHA_FALLECIMIENTO

In [9]:
# Fecha del primer fallecido reportado
date_a = pd.to_datetime(df_fal['FECHA_FALLECIMIENTO'], format = '%Y%m%d').dt.date
date_a = date_a.min()
print(f'Fecha del primer fallecido reportado: {date_a}')
del date_a

# Fecha del último fallecido reportado
date_b = pd.to_datetime(df_fal['FECHA_FALLECIMIENTO'], format = '%Y%m%d').dt.date
date_b = date_b.max()
print(f'Fecha del último fallecido reportado: {date_b}')
del date_b

Fecha del primer fallecido reportado: 2020-03-03
Fecha del último fallecido reportado: 2022-01-01


#### 1.3. EDAD_DECLARADA

In [11]:
fal_edad = df_fal['EDAD_DECLARADA'].value_counts()

print('Edad (años): no_casos \nDesde la edad más común de fallecimientos a la menos común.\n')
lst = []

for idx, edad in enumerate(fal_edad):
    index = fal_edad.index
    lst.append(f'{index[idx]}: {edad}')
    
chunks = [lst[x:x+20] for x in range(0, len(lst), 20)]
df = pd.DataFrame.from_records(chunks)
df = df.transpose()
print(df.to_string(index=False, header=False))

del fal_edad, chunks, df, lst, edad, idx, index

Edad (años): no_casos
Desde la edad más común de fallecimientos a la menos común.

66: 5501 80: 4144 48: 2221 34: 642   1: 149 102: 40
67: 5407 58: 4027 89: 2085 33: 625  20: 113   5: 40
65: 5394 79: 4020 47: 2029 95: 566  19: 111   6: 37
68: 5356 57: 4003 90: 1870 32: 555 100: 106 103: 21
70: 5255 82: 3856 46: 1855 96: 459   18: 89 104: 16
72: 5200 81: 3823 45: 1780 31: 435    2: 83  106: 9
73: 5188 56: 3714 44: 1679 30: 426   17: 77  108: 6
69: 5164 55: 3694 43: 1504 29: 347   16: 77  105: 5
64: 5100 83: 3566 91: 1446 27: 317  101: 71  107: 3
74: 5084 54: 3442 42: 1357 28: 315    4: 63  109: 2
71: 5065 84: 3399 92: 1351 97: 309   15: 56  113: 2
63: 5055 85: 3217 41: 1268 26: 269   14: 53  110: 2
61: 5047 53: 3112 40: 1094 98: 245   11: 52   -9: 1
75: 4935 86: 2845 39: 1064 25: 244    8: 52  117: 1
62: 4909 52: 2810 93: 1018 24: 204    9: 52  114: 1
76: 4808 51: 2712  38: 975 99: 191    3: 51    None
60: 4594 87: 2668  37: 834 23: 189   10: 50    None
77: 4588 50: 2567  94: 760 21: 17

Existen valores negativos, necesario correguir. Edad mínima 0 años y edad máxima 117 años.

#### 1.4. SEXO

In [13]:
fal_sexo = df_fal['SEXO'].value_counts().sort_index()

for idx, sex in enumerate(fal_sexo):
    index = fal_sexo.index
    print(f'{index[idx]}: {sex}')

del fal_sexo, sex, idx, index

.: 3
FEMENINO: 73685
MASCULINO: 129094


Hay puntos en lugar de vacíos.

#### 1.5. CLASIFICACION_DEF

In [15]:
fal_cla = df_fal['CLASIFICACION_DEF'].value_counts().sort_index()

for idx, clase in enumerate(fal_cla):
    index = fal_cla.index
    print(f'{index[idx]}: {clase}')

del fal_cla, clase, idx, index

Criterio SINADEF: 57884
Criterio clÃ­nico: 9756
Criterio investigaciÃ³n EpidemiolÃ³gica: 16078
Criterio nexo epidemiolÃ³gico: 2951
Criterio radiolÃ³gico: 6718
Criterio serolÃ³gico: 44532
Criterio virolÃ³gico: 64863


#### 1.6. DEPARTAMENTO

In [16]:
fal_dep = df_fal['DEPARTAMENTO'].value_counts().sort_index()

for idx, dep in enumerate(fal_dep):
    index = fal_dep.index
    print(f'{index[idx]}: {dep}')
print(f'\nTotal fallecidos: {fal_dep.sum()}')

del fal_dep, dep, idx, index

AMAZONAS: 1288
ANCASH: 6771
APURIMAC: 1526
AREQUIPA: 9765
AYACUCHO: 2162
CAJAMARCA: 4193
CALLAO: 10123
CUSCO: 4838
HUANCAVELICA: 1183
HUANUCO: 2721
ICA: 8612
JUNIN: 7106
LA LIBERTAD: 10400
LAMBAYEQUE: 8691
LIMA: 89521
LORETO: 4283
MADRE DE DIOS: 777
MOQUEGUA: 1537
PASCO: 1056
PIURA: 12279
PUNO: 4264
SAN MARTIN: 3048
TACNA: 1978
TUMBES: 1595
UCAYALI: 3065

Total fallecidos: 202782


#### 1.7. PROVINCIAS

In [17]:
fal_pro = df_fal['PROVINCIA'].value_counts().sort_index()
print(f'Igual que la variable departamento pero con {len(fal_pro)} provincias.')
del fal_pro

Igual que la variable departamento pero con 196 provincias.


#### 1.8. DISTRITO

In [18]:
fal_dis = df_fal['DISTRITO'].value_counts().sort_index()
print(f'Igual que la variable departamento pero con {len(fal_dis)} distritos.')
del fal_dis

Igual que la variable departamento pero con 1646 distritos.


#### 1.9. UBIGEO

In [19]:
fal_ubi = df_fal['UBIGEO'].value_counts().sort_index()
print(f'{len(fal_ubi)} códigos de UBIGEO, para localizar zonas.')
del fal_ubi

1781 códigos de UBIGEO, para localizar zonas.


#### 1.10. id_persona

In [25]:
df_fal['id_persona'] = df_fal['id_persona'].fillna(0)
df_fal['id_persona'] = df_fal['id_persona'].apply(np.int64)

print('ID únicos para identificar a cada persona: ')
print(df_fal['id_persona'].head().to_string(header=False))

ID únicos para identificar a cada persona: 
0    25819410
1    24693187
2    24693271
3    24735448
4    24786629


In [26]:
del df_fal

## 2. Cargar el dataset de vacunados contra COVID-19

### Cargar usando dask, debido al tamaño del csv

In [4]:
url_vac = 'RawData/TB_VACUNACION_COVID19.csv'

start = time.time()
df_vac = dd.read_csv(url_vac, sep = ",", dtype = {'fecha_vacunacion':'int32', 'id_eess': 'int32',
                                                  'dosis': 'category','id_centro_vacunacion': 'category',
                                                  'id_vacuna': 'category','id_grupo_riesgo': 'category', 
                                                  'edad':'float64'})
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

del url_vac, start, end
df_vac.head()

Read csv with dask:  0.02258753776550293 sec


,id_persona,id_vacunados_covid19,fecha_vacunacion,id_eess,id_centro_vacunacion,id_vacuna,id_grupo_riesgo,dosis,edad
0,5619296,27392550,20211211,26148,154,1,68,1,36.0
1,5736337,27393830,20210922,8915,3361,6,68,2,33.0
2,9191009,39824666,20211217,25811,1813,5,77,2,16.0
3,11009648,45081339,20211123,26478,2968,1,71,1,21.0
4,9280212,38775211,20211116,15610,0,6,71,1,28.0


### Porcentaje de datos perdidos de cada variable

In [69]:
fn.missing_values(df_vac)

,index,percent_missing
0,0,0.000000
1,1,0.000000
2,2,0.000000
3,3,0.000000
4,4,0.000000
5,5,0.000000
6,6,0.000000
7,7,0.000000
8,8,0.008505


### Analizar cada variable del dataset de vacunados

#### 2.1. id_persona

In [3]:
print('ID únicos para identificar a cada persona:')
df_vac['id_persona'].head()

ID únicos para identificar a cada persona:


0    8255057
1    5239468
2    5922737
3    1139188
4    1076540
Name: id_persona, dtype: int64

#### 2.2. id_vacunados_covid19

In [30]:
print('ID únicos para identificar a cada vacunado (como un contador de cada vacuna aplicada):')
df_vac['id_vacunados_covid19'].head()

ID únicos para identificar a cada vacunado (como un contador de cada vacuna aplicada):


0    45853675
1    31291708
2    33782258
3     3818412
4     3819052
Name: id_vacunados_covid19, dtype: int64

#### 2.3. fecha_vacunacion

In [3]:
date_a = df_vac['fecha_vacunacion'].min().compute()
date_a = pd.to_datetime(date_a, format = '%Y%m%d')

date_b = df_vac['fecha_vacunacion'].max().compute()
date_b = pd.to_datetime(date_b, format = '%Y%m%d')

print(f'Fecha de inicio de vacunación: {date_a} \nFecha última vacunación: {date_b}')
del date_a, date_b

Fecha de inicio de vacunación: 2020-04-27 00:00:00 
Fecha última vacunación: 2022-01-03 00:00:00


#### 2.4. id_eess

In [4]:
print(f'id_iess\tnumber')
df_vac.id_eess.value_counts().compute().sort_index()

id_iess	number


33          36
49         220
51        1419
52        2143
53          63
         ...  
28772      731
28780       23
28791      611
28806    11035
28820      691
Name: id_eess, Length: 7875, dtype: int64

#### 2.5. id_centro_vacunacion	

In [38]:
print(f'id_centro_vacunacion\tnumber')
df_vac.id_centro_vacunacion.value_counts().compute().sort_index()

id_centro_vacunacion	number


0       14609823
1           1924
3         453905
5          24562
6         103575
          ...   
4986         316
4987           6
4993          93
4997           6
4998          15
Name: id_centro_vacunacion, Length: 3436, dtype: int64

#### 2.6. id_vacuna

In [39]:
print(f'id_vacuna\tnumber')
df_vac.id_vacuna.value_counts().compute().sort_index()

id_vacuna	number


1     3155019
2      371422
3      396775
4      700791
5    26693862
6    18512653
Name: id_vacuna, dtype: int64

#### 2.7. id_grupo_riesgo

In [41]:
df_vac.id_grupo_riesgo.value_counts().compute().sort_index()

0        3909
1         207
4       30366
6        2294
7      315102
11     226950
12     334325
17    1535045
22     111433
23     414681
24       1657
26       4043
36      40348
40     343179
45      16969
47      18389
50      17481
51       9817
52       1165
53      18131
54    9439904
56      28974
57     106056
58     110514
59       3061
60      30100
61      11236
62    5945090
63      36410
64      70599
65      44686
66     255361
67    7387919
68    8056930
69     129794
70    1658634
71    8339201
72       4924
73       4757
74       3213
75       4633
76      17447
77    4694589
78        918
79         81
Name: id_grupo_riesgo, dtype: int64

#### 2.8. dosis

In [4]:
print(f'dosis\tnumber')
df_vac.dosis.value_counts().compute()

dosis	number


1    24455931
2    21472095
3     3902389
4         101
5           6
Name: dosis, dtype: int64

#### 2.9. edad

In [5]:
df_vac.edad.value_counts().compute().sort_index()

0.0      3
1.0      2
2.0      4
3.0      2
4.0      7
        ..
141.0    2
143.0    2
146.0    2
147.0    1
224.0    2
Name: edad, Length: 136, dtype: int64

## 3. Cargar el dataset de casos positivos de COVID-19

### Cargar el dataset

In [3]:
url_cas = 'RawData/positivos_covid.csv'
df_cas = pd.read_csv('RawData/positivos_covid.csv', sep = ';')

del url_cas
df_cas.head()

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona
0,20220101,LIMA,LIMA,SAN MARTIN DE PORRES,PR,25.0,MASCULINO,20201217.0,150135.0,24662153.0
1,20220101,ICA,PISCO,PISCO,PR,20.0,FEMENINO,20200822.0,110501.0,24662175.0
2,20220101,HUANUCO,HUANUCO,HUANUCO,PR,22.0,FEMENINO,20200729.0,100101.0,24662197.0
3,20220101,ANCASH,SANTA,SANTA,AG,18.0,FEMENINO,20210630.0,21808.0,24662204.0
4,20220101,ANCASH,SANTA,NUEVO CHIMBOTE,AG,17.0,MASCULINO,20210404.0,21809.0,24662207.0


### Porcentaje de datos perdidos de cada variable

In [5]:
fn.missing_values(df_cas)

,index,percent_missing
0,FECHA_CORTE,0.000000
1,DEPARTAMENTO,0.000000
2,PROVINCIA,0.000000
3,DISTRITO,0.000000
4,METODODX,0.000000
5,EDAD,0.015069
6,SEXO,0.000043
7,FECHA_RESULTADO,0.087854
8,UBIGEO,4.884530
9,id_persona,1.752829


### Analizar cada variable del dataset de casos reportados